# Où suis-je ?

In [ ]:
pwd()

# Se positionner dans son répertoire de travail (dossier de téléchargement et/ou google drive)

In [ ]:
cd("G:\\My Drive\\")

# Listing des paquets disponibles dans l'environnement actuel 
Si la librairie LibFEM est absente on peut utiliser les 2 commandes suivantes indiféremmen using Pkg;
Pkg.add(PackageSpec(url="https://github.com/amdeld/LibFEM.jl")) ou 
]add https://github.com/amdeld/LibFEM.jl
 

In [ ]:
]st


# On charge ensuite la librairie LibFEM avec la commande suivante:

In [ ]:
using LibFEM # ,Plots

# Quelques informations quant au cas à traiter

In [ ]:
#PARAMETERS
const L=10000. #length in mm
const A=100. #cross-sectional area in mm^2
const E=210000. #modulus of elasticity in MPa [steel]
const FM=10000.; #force modulus in N

In [ ]:
# ===============================================PRE-PROCESSING==================
# DEFINING AND DISCRETIZING[MESHING] THE STRUCTURE
# connectivity table
# elt||node_i||node_j
# 1|1|3
# 2|2|3
# grid()
X1pos=0.;Y1pos=0.
X2pos=0.;Y2pos=L
X3pos=L;Y3pos=L
#lengths
L1=D2_TrussElementLength(X1pos,Y1pos,X3pos,Y3pos) #length of element 1
L2=D2_TrussElementLength(X2pos,Y2pos,X3pos,Y3pos) #length of element 2
#APPLYING GEOMETRIC&MATERIAL PROPERTIES
A1=sqrt(2)*A; #cross-sectional area of element 1
A2=A; #cross-sectional area of element 2
E1=E; #material of element 1
E2=E; #material of element 2

# Calcul et Affichage des matrices de raideur élémentaires

In [ ]:
#writing-defining the element stiffness matrices
K1=D2_TrussElementStiffness(E1,A1,L1,45);
println("K1=\r")
K1

In [ ]:
K2=D2_TrussElementStiffness(E2,A2,L2,0);
println("K2=\r")
K2

# Affichage des matrices de raideurs positionnées $[K_1]^p$ $[K_2]^p$ et et de la matrice d'assemblage $[K]$

In [ ]:
#ASSEMBLING THE GLOBAL STIFFNESS MATRIX
#matrices initialization
K=zeros(6,6);K1P=zeros(6,6);
#positionning stiffness matrices
K1P=D2_TrussAssemble(K,K1,1,3)
println("K1P=\r")
K1P

In [ ]:
K=zeros(6,6);K2P=zeros(6,6);
K2P=D2_TrussAssemble(K,K2,2,3)
println("K2P=\r")
K2P

In [ ]:
#assembling
K=K1P+K2P
println("K=\r")
K

# Affichage des vecteurs des déplacements et des chargements nodaux

In [ ]:
#SOLVING DISPACEMENT EQUATIONS
#extracting displacement submatrix via index vector
K_s=K[5:6,5:6]
#Setting-up the force subvector by applying Load & Boundary Conditions[LBC]]
F_s=[0 FM]'
#solving by gaussian elimination
U_s=K_s\F_s
#SOLVING FORCE EQUATIONS
#setting-up the global nodal displacement vector
U=[0 0 0 0 U_s[1:2]']'
println("U=\r")
U

In [ ]:
#computing the global nodal force vector
F=K*U
println("F=\r")
F

# Calcul et Affichage des grandeurs dérivées déformations et contraintes élémentaires

In [ ]:
#COMPUTING STRESSES
#writing the element nodal displacement vectors
U1=[U[1] U[2] U[5] U[6]]'
U2=[U[3] U[4] U[5] U[6]]'
#computing element strains
ϵ1=D2_TrussElementStrain(L1,45,U1)
println("ϵ1=\r")
ϵ1

In [ ]:
ϵ2=D2_TrussElementStrain(L2,0,U2)
println("ϵ2=\r")
ϵ2

In [ ]:
#computing element forces
f1=D2_TrussElementForce(E1,A1,L1,45,U1)
f2=D2_TrussElementForce(E2,A2,L2,0,U2)
#computing element stresses
σ1=D2_TrussElementStress(E1,L1,45,U1)
println("σ1=\r")
σ1

In [ ]:
σ2=D2_TrussElementStress(E2,L2,0,U2)
println("σ2=\r")
σ2